In [ ]:
import torch

assert torch.cuda.is_available(), "Cuda is not available"
dev=torch.cuda.current_device()
print ("Cuda device name: ", torch.cuda.get_device_name (dev))

Cuda device name:  Tesla K80


In [ ]:
from torchvision.models.inception import inception_v3
import contextlib
import os

with contextlib.redirect_stdout (open(os.devnull, 'w')):
  model=inception_v3 (pretrained=True, transform_input=True).to(dev)
  model.aux_logits=False
  model.train(False)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

In [ ]:
#Dog vs Cat images

with contextlib.redirect_stdout (open(os.devnull, 'w')):
  !wget -nc https://www.dropbox.com/s/ae1lq6dsfanse76/dogs_vs_cats.train.zip?dl=1 -O data.zip
  !unzip -n data.zip

print ("data.zip is ready at train")

data.zip is ready at train


In [ ]:
from copy import deepcopy
embedding = deepcopy (model)

class Idendity (torch.nn.Module):
  def __init__ (self):
    super(Idendity, self).__init__()
  
  def forward (self, x):
    return x

embedding.fc=Idendity().to(dev)

In [ ]:
from tqdm import tqdm
from imageio import imread
import PIL.Image as Image
import numpy as np

X, Y= [], []
batch_size=len(os.listdir ("train"))//1000
imgs=np.zeros ([batch_size, 299, 299, 3])
batch_index=0
for fname in tqdm (os.listdir("train")):
  y=fname.startswith("cat")
  Y.append(y)
  
  img=imread (os.path.join("train", fname))
  img=np.array (Image.fromarray(img).resize((299, 299)))/255
  imgs[batch_index]=img

  if batch_index==batch_size-1:
    input_tensor=torch.as_tensor (imgs.transpose ([0, 3, 1, 2]), dtype=torch.float32).to(dev)

    with torch.no_grad():
      features = embedding (input_tensor).to(dev)

    X.append(features)

    batch_index=0

    continue
  batch_index+=1

  0%|          | 22/25000 [00:00<03:50, 108.45it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 25000/25000 [04:19<00:00, 96.44it/s]


In [ ]:
Xx=np.concatenate ([i.cpu() for i in X])

Yy=np.array(Y[:len(Xx)]).astype (int)

print (Xx.shape, Yy.shape)

(25000, 2048) (25000,)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split (Xx, Yy, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val=train_test_split (X_train, y_train, test_size=0.25, random_state=42)
print ("X_train, y_train shape: ", X_train.shape, y_train.shape)
print ("X_val, y_val shape: ", X_val.shape, y_val.shape)
print ("X_test, y_test shape: ", X_test.shape, y_test.shape)

X_train, y_train shape:  (15000, 2048) (15000,)
X_val, y_val shape:  (5000, 2048) (5000,)
X_test, y_test shape:  (5000, 2048) (5000,)


In [ ]:
def iterate_mini_batches (X, y, batch_size):
  indices=np.random.permutation(np.arange (len(X)))
  for i in range (0, len(X), batch_size):
    batch=indices[i : i+batch_size]
    yield X[batch], y[batch]

In [ ]:
# <YOUR CODE>
from torch.nn import LazyLinear, Sequential, ReLU, BCELoss, BCEWithLogitsLoss, BatchNorm1d, Dropout, Linear
import torch.nn.functional as F

class Head (torch.nn.Module):
  def __init__ (self):
    bias=True
    super (Head, self).__init__()
    self.first=LazyLinear (1024, bias=bias)
    self.second=LazyLinear (64, bias=bias)
    self.last=LazyLinear (1, bias=bias)

    self.relu=ReLU()
    self.batchnorm1=BatchNorm1d(1024)
    self.batchnorm2=BatchNorm1d(64)
    self.dropout=Dropout (0.1)
  
  def forward (self, inputs):
    inputs=self.relu (self.first(inputs))
    # inputs=self.batchnorm1 (inputs)
    inputs=self.relu (self.second (inputs))
    # inputs=self.batchnorm2 (inputs)
    inputs=self.dropout (inputs)
    inputs=self.last(inputs)

    return inputs

In [ ]:
import time

bias=False
layers=[128, 64, 1]
num_epochs=10
batch_size=100
learning_rate=0.001
mu=0.99
head=Head()
head.to(dev)
optimizer=torch.optim.Adam(head.parameters(), lr=learning_rate)
train_loss, val_accuracy=[], []


for epoch in range (num_epochs):
  head.train(True)
  start_time=time.time()
  for X_batch, y_batch in iterate_mini_batches (X_train, y_train, batch_size):
    X_batch=torch.as_tensor (X_batch, dtype=torch.float32).to(dev)
    # y_batch=np.concatenate (([y_batch], [np.logical_not (y_batch).astype(int)]), axis=0).T
    y_batch=torch.as_tensor (y_batch, dtype=torch.float32).to(dev)
    output=head(X_batch)
    # print (output[:5])
    # print (y_batch.reshape(-1, 1)[:5])
    # loss=F.cross_entropy (output, y_batch).mean()
    loss=BCEWithLogitsLoss()(output[:, 0], y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    train_loss.append (loss.cpu().data.numpy())
  
  torch.save(head.state_dict(), "head.ckpt")

  head.train(False)
  for X_batch, y_batch in iterate_mini_batches (X_val, y_val, batch_size):
    X_batch=torch.as_tensor (X_batch, dtype=torch.float32).to(dev)
    output=torch.round(torch.sigmoid (head(X_batch))).data.cpu().numpy()
    val_accuracy.append (np.mean(y_batch==output[:, 0]))

  
  print("Epoch {} of {} took {:.3f}s".format(
      epoch + 1, num_epochs, time.time() - start_time))
  print("  training loss (in-iteration): \t{:.6f}".format(
      np.mean(train_loss[-len(X_train) // batch_size :])))
  print("  validation accuracy: \t\t\t{:.2f} %".format(
      np.mean(val_accuracy[-len(X_val) // batch_size :]) * 100))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Epoch 1 of 100 took 0.716s
  training loss (in-iteration): 	0.061438
  validation accuracy: 			98.78 %
Epoch 2 of 100 took 0.666s
  training loss (in-iteration): 	0.036116
  validation accuracy: 			98.80 %
Epoch 3 of 100 took 0.662s
  training loss (in-iteration): 	0.026139
  validation accuracy: 			98.84 %
Epoch 4 of 100 took 0.660s
  training loss (in-iteration): 	0.021802
  validation accuracy: 			98.98 %
Epoch 5 of 100 took 0.658s
  training loss (in-iteration): 	0.020355
  validation accuracy: 			98.94 %
Epoch 6 of 100 took 0.649s
  training loss (in-iteration): 	0.019189
  validation accuracy: 			98.80 %
Epoch 7 of 100 took 0.651s
  training loss (in-iteration): 	0.018828
  validation accuracy: 			98.86 %
Epoch 8 of 100 took 0.644s
  training loss (in-iteration): 	0.015114
  validation accuracy: 			98.70 %
Epoch 9 of 100 took 0.655s
  training loss (in-iteration): 	0.012152
  validation accuracy: 			98.92 %
Epoch 10 of 100 took 0.655s
  training loss (in-iteration): 	0.018257
  v

In [ ]:
head.train(False)
test_accuracy=[]
for X_batch, y_batch in iterate_mini_batches (X_test, y_test, batch_size):
  X_batch=torch.as_tensor (X_batch, dtype=torch.float32).to(dev)
  output=torch.round(torch.sigmoid (head(X_batch))).data.cpu().numpy()
  acc=np.mean (output[:, 0]==y_batch)
  test_accuracy.append (acc)

test_acc=np.mean (test_accuracy)

print (f"Test Accuracy: {test_acc}")

Test Accuracy: 0.9852
